In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
main_data_raw = pd.read_csv('data\sap_storing_data_hu_project.csv', engine='pyarrow')
main_data_raw.head()
main_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898526 entries, 0 to 898525
Columns: 140 entries,  to stm_x_vervallen_ind
dtypes: float64(22), int64(8), object(110)
memory usage: 959.7+ MB
